## Install/import packages

In [1]:
!pip install pandas
!pip install langchain

import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Load the unlabeled data into a dataframe

In [2]:
leadership_df = pd.read_csv('Mentor.Leadership.By.Semester.Combined.csv')

leadership_df

,Response,Semester
0,"Although Owens leadership skills are good, he ...",Fall 2017
1,Andrew should utilize colleagues to generate i...,Fall 2017
2,Bailey worked one part-time internship and her...,Fall 2017
3,BECAUSE I DIDN'T ALLOW FOR ANY EXPOSURE FOR CH...,Fall 2017
4,Ben's greatest area for improvement is in the ...,Fall 2017
...,...,...
1311,Only choosing this because unfortunately she w...,Fall 2022
1312,Sara could benefit from recognizing her worth ...,Fall 2022
1313,There was not much room to use leadership skills.,Fall 2022
1314,We do not have many leadership opportunities i...,Fall 2022


## Load the labeled data

In [18]:
labeled_df = pd.read_csv('Labelbox Cleaned-2020-10-22.csv')
labeled_df.rename(columns={'Final Label': 'Label'}, inplace=True)

labeled_df

,Labeled Data,Label,Dataset Name
0,NaN,NONE,Student
1,\n,NONE,Mentor
2,And then Alex can weigh in his ideas.,NONE,Student
3,"As Ally's career progresses, I have no doubt ...",NONE,Mentor
4,Caleb did a lot of the work filming and editi...,NONE,Student
...,...,...,...
6154,when discussing new project ideas for the futu...,STC,Student
6155,will also additionally take more responsibilit...,NONE,Student
6156,worked collaboratively with the study abroad o...,NONE,Student
6157,worked solo\n,NONE,Mentor


## Remove "Dataset Name" column from dataframe

In [19]:
labeled_df.drop(columns='Dataset Name', inplace=True)
labeled_df

,Labeled Data,Label
0,NaN,NONE
1,\n,NONE
2,And then Alex can weigh in his ideas.,NONE
3,"As Ally's career progresses, I have no doubt ...",NONE
4,Caleb did a lot of the work filming and editi...,NONE
...,...,...
6154,when discussing new project ideas for the futu...,STC
6155,will also additionally take more responsibilit...,NONE
6156,worked collaboratively with the study abroad o...,NONE
6157,worked solo\n,NONE


## Parse labeled responses for 2 of each label into JSON format

In [35]:
def parse_responses_into_json():
    # Sample 2 rows for each unique label in the 'Label' column
    sampled_df = labeled_df.groupby('Label').apply(lambda x: x.sample(2)).reset_index(drop=True)

    # Convert df to JSON string
    json_responses = sampled_df.to_json(orient='records')

    return json_responses

res = parse_responses_into_json()
print(res)

[{"Labeled Data":"At the beginning of the semester I had the opportunity to lead as I was given the responsibility to inform her of various things that were happening in our programs as well as teach her how to work our database that we use for participant data.\n","Label":"IDK"},{"Labeled Data":"Coming into CU-ICAR, I felt that their social media was lacking in a few areas.","Label":"IDK"},{"Labeled Data":"I don't know that he doesn't have this leadership skill, it was just not demonstrated in the internship.","Label":"NONE"},{"Labeled Data":"I think this can be an area that he could work on.\n","Label":"NONE"},{"Labeled Data":"I had to make sure current Scholars were available and willing to assist with logistical aspects of the weekend. \n","Label":"STC"},{"Labeled Data":"These individual assignments are assigned by my boss based on our strengths, technical expertise, and experience in the subject.","Label":"STC"},{"Labeled Data":"When he works on projects for his team, he often ask

## Generate prompt

In [40]:
def generate_label_prompt(text):
    # Filter for 2 of each label
    labeled_data_json = parse_responses_into_json()

    combined_json_str = '[' + ', '.join(labeled_data_json) + ']'

    labeled_data = "# Labeled Data\n" + labeled_data_json + "\n"

    instructions = "# Instructions\n\"Based on the text in the 'Text' section below that contains an employer's evaluation of their intern's leadership skills, determine whether the text indicates a more transactional or transformational leadership style based on the definitions provided below in the 'Definitions' section. Labeled JSON data is provided in the 'Labeled Data' section as a reference for accurate labels of previous text examples. Give the output in a JSON format with 2 key value pairs where the 'Explanation' key corresponds to your reasoning for your choice and the 'Label' key corresponds to your choice for whether the text shows 'Transactional' or 'Transformational' leadership.\"\n"
    text = "# Text\n\"" + text + "\""
    definitions = "# Definitions\n\"'Transactional leadership: characterized by a focus on supervision, organization, and performance. Leaders who exhibit this style often work within established procedures and promote compliance through rewards and punishments.' 'Transformational leadership: Transformational leaders inspire their followers to exceed their own self-interests for the good of the group or organization. They often encourage innovation, creativity, and personal development. They stimulate and inspire followers via charisma to both achieve extraordinary outcomes and develop their own leadership capacity.'\"\n"
    new_prompt = instructions + definitions + labeled_data + text
    return new_prompt

example = generate_label_prompt(leadership_df["Response"][1])
print(example)

# Instructions
"Based on the text in the 'Text' section below that contains an employer's evaluation of their intern's leadership skills, determine whether the text indicates a more transactional or transformational leadership style based on the definitions provided below in the 'Definitions' section. Labeled JSON data is provided in the 'Labeled Data' section as a reference for accurate labels of previous text examples. Give the output in a JSON format with 2 key value pairs where the 'Explanation' key corresponds to your reasoning for your choice and the 'Label' key corresponds to your choice for whether the text shows 'Transactional' or 'Transformational' leadership."
# Definitions
"'Transactional leadership: characterized by a focus on supervision, organization, and performance. Leaders who exhibit this style often work within established procedures and promote compliance through rewards and punishments.' 'Transformational leadership: Transformational leaders inspire their follower